In [15]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import pickle

In [16]:
df = pd.read_csv('data.csv', header=0)

In [17]:
# nuuksio area code
df = df[df['ASTA_Counters.RegionalEntity']==625046]

In [18]:
df = df.drop('CounterReadingID', 1)
df = df.drop('CounterID_ASTA', 1)
df = df.drop('SequenceNumber', 1)
df = df.drop('ASTA_Counters.InstallationDate', 1)
df = df.drop('ASTA_Counters.NationalParkCode', 1)
df = df.drop('ASTA_Counters.RegionalUnit', 1)
df = df.drop('ASTA_Counters.RegionalEntity', 1)
df = df.drop('ASTA_Counters.Name_ASTA', 1)
df = df.drop('PAVE_Counters.Globalid', 1)
df = df.drop('PAVE_Counters.Manager', 1)
df = df.drop('PAVE_Counters.AdditionalInfo', 1)
df = df.drop('PAVE_Counters.CoordinateNorth', 1)
df = df.drop('PAVE_Counters.CoordinateEast', 1)
df = df.drop('ASTA_Counters.Municipality', 1)
df = df.drop('Unnamed: 0', 1)

In [19]:
df = df.rename({'ASTA_Counters.CounterID_PAVE': 'CounterId', 'PAVE_Counters.Name': 'CounterName'}, axis=1)

In [20]:
df['Weekday'] = [None for x in range(len(df))]
df['Month'] = [None for x in range(len(df))]
df['Hour'] = [None for x in range(len(df))]

In [21]:
def f(row):
    timestamp = datetime.strptime(row['StartTime'], '%Y-%m-%d %H:%M:%S')
    row['Weekday'] = timestamp.weekday()
    row['Hour'] = timestamp.hour
    row['Month'] = timestamp.month
    return row

In [22]:
df = df.apply(f, axis=1)

In [23]:
df['Weekday'] = df['Weekday'].astype('category')
df['Month'] = df['Month'].astype('category')
df['CounterId'] = df['CounterId'].astype('int64')
df['CounterId'] = df['CounterId'].astype('category')

In [24]:
df = pd.concat([df, pd.get_dummies(df['Weekday'], prefix='Weekday')], axis=1)
df = pd.concat([df, pd.get_dummies(df['Month'], prefix='Month')], axis=1)
df = pd.concat([df, pd.get_dummies(df['CounterId'], prefix='CounterId')], axis=1)

In [25]:
for x in df['CounterId'].unique():
    print(x)
    print(df[df['CounterId'] == x]['CounterName'].unique()[0])

149157
Haukkalampi - Solvalla yhdysreitin kävijälaskuri
151989
Nuuksio, Veikkolan kävijälaskuri
186819
Mustakorventien 2. laskuri
188852
Siikaranta kävijälaskuri
109144
Kattila niityn portin kävijälaskuri
125383
Nuuksio Takalanpolun laskuri
94722
Klassarinkierroksen kävijälaskuri
51257
Haukkalammen ekopisteen kävijälaskuri
168237
Matalajärven kävijälaskuri


In [26]:
df = df.drop('StartTime', 1)
df = df.drop('EndTime', 1)
df = df.drop('CounterName', 1)
df = df.drop('CounterId', 1)
df = df.drop('Weekday', 1)
df = df.drop('Month', 1)

In [27]:
y = np.array(df['Visits'])

X = df.drop('Visits', axis=1)

X_names = list(X.columns)

X = np.array(X)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

In [29]:
rf = RandomForestRegressor(n_estimators = 1500, max_depth=15, random_state = 1)

rf.fit(X_train, y_train);

In [30]:
y_pred = rf.predict(X_test)

errors = abs(y_pred - y_test)

print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 6.19 degrees.


In [31]:
score = rf.score(X_test, y_test)

print('Accuracy:', round(score * 100, 2), '%.')

Accuracy: 49.77 %.


In [32]:
pickle.dump(rf, open('nuuksio_paths.sav', 'wb'))